In [10]:

# NLDM=open(NLDM_file_name,'r')

In [11]:
# print(NLDM.read())

In [5]:
def delayModelFileParsing(fileName):
#     try:
    NLDM=open(fileName,'r')
        
    initialCase=1
    gateName=""
    valueCounter=0
    templist_DT=[]
    templist_OS=[]
    for line in NLDM.readlines():
        match initialCase:
            case 1:
                if("cell (" in line):
                    gateName=line.split("(")[1].split(")")[0]
                    gateInformation[str(gateName)]={}
                    initialCase=2
                    continue
            case 2:
                if("capacitance		:" in line):
                    capVal=float(line.partition(":")[2].split(";")[0])
                    gateInformation[str(gateName)]["inputCapacitance"]=capVal
                    initialCase=3
                    continue
            case 3:
                if "cell_delay(Timing_7_7)" in line:
                    initialCase=4
                    continue
            case 4:
                if "index_1" in line:
                    templist=line.split("(")[1].split(")")[0].replace('"','').split(",")
                    for i in range(0,len(templist)):
                        templist[i]=float(templist[i])
                    gateInformation[str(gateName)]["inputSlew"]=templist
                    initialCase=5
                    continue
            case 5:
                if "index_2" in line:
                    templist2=line.split("(")[1].split(")")[0].replace('"','').split(",")
                    for i in range(0,len(templist2)):
                        templist2[i]=float(templist2[i])
                    gateInformation[str(gateName)]["loadCapacitance"]=templist2
                    initialCase=6
                    continue
            case 6:
                if(valueCounter<7):
                        templist3=line.split('\"')[1].split(",")

                        for i in range(0,len(templist3)):
                            templist3[i]=float(templist3[i])
                        templist_DT.append(templist3)
                        valueCounter+=1
                        initialCase=6
                else:
                    gateInformation[str(gateName)]["delayTable"]=templist_DT
                    valueCounter=0
                    initialCase=7
            case 7:
                if "output_slew(Timing_7_7)" in line:
                    initialCase=8
                    continue
            case 8:
                if "index_1" in line:
                    initialCase=9
                    continue
            case 9:
                if "index_2" in line:
                    initialCase=10
                    continue
            case 10:
                if(valueCounter<7):
                        templist4=line.split('\"')[1].split(",")
                        for i in range(0,len(templist4)):
                            templist4[i]=float(templist4[i])
                        templist_OS.append(templist4)
                        valueCounter+=1
                        initialCase=10
                else:
                    gateInformation[str(gateName)]["outputSlew"]=templist_OS
                    valueCounter=0
                    initialCase=1
                    templist_DT=[]
                    templist_OS=[]
            
            

In [2]:
NLDM_file_name="NLDM_lib_max2Inp"
gateInformation={}
circuitInfo={}
delayModelFileParsing(NLDM_file_name)

In [6]:
def circuitFileParsing(circuitFile):
    
#     try:
        ckt=open(circuitFile,'r')
        print("Parsing Circuit file : "+ circuitFile)
        for line in ckt.readlines():
            if len(line.split()) == 0:
                continue
            elif("#" not in line):
                if "=" in line:
                    if "DFF" in line:
                        lineSplit=line.split("=")
                        gateNo=str(lineSplit[0].strip())
                        if gateNo not in circuitInfo:
                            circuitInfo[gateNo]={}
                            

                        circuitInfo[gateNo]["isInput"]=True
                        circuitInfo[gateNo]["isOutput"]=False
                        gateName=str(lineSplit[1].split("(")[0].strip())
                        circuitInfo[gateNo]["gateType"]="INPUT"



                        tempOutputNode=lineSplit[1].split("(")[1].split(")")[0].strip()
                        circuitInfo[tempOutputNode]={}
                        circuitInfo[tempOutputNode]["isOutput"]=True
                        circuitInfo[tempOutputNode]["gateType"]="OUTPUT"
                        circuitInfo[tempOutputNode]["isInput"]=False
                        outputsforAT.append(tempOutputNode)

                    else:
                        lineSplit=line.split("=")
                        gateNo=str(lineSplit[0].strip())
                        gateName=str(lineSplit[1].split("(")[0].strip())
                        fanIns=[k.strip() for k in lineSplit[1].split("(")[1].split(")")[0].split(",")]

                        for i in fanIns:
                            if(i  not in circuitInfo):
                                circuitInfo[i]={}
                            
                            if "fanOut" not in circuitInfo[i]:
                                circuitInfo[i]["fanOut"]=[]
                            if isinstance(circuitInfo[i]["fanOut"],list):
                                dummy=circuitInfo[i]["fanOut"]
                                if gateNo not in dummy:
                                    circuitInfo[i]["fanOut"].append(gateNo)
                            else:
                                circuitInfo[i]["fanOut"]=[]
                                circuitInfo[i]["fanOut"].append(gateNo)
#                             else:
#                                 circuitInfo[i]["fanOut"].append(gateNo)
                        if gateNo not in circuitInfo:
                            circuitInfo[gateNo]={}
                        circuitInfo[gateNo]["gateType"]=gateName
                        circuitInfo[gateNo]["fanIn"]=fanIns
                elif ("=" not in line):
                    gateNo=str(line.split("(")[1].replace(" )","").replace("\n","").strip().upper())
                    if gateNo in circuitInfo:
                        continue
                    elif gateNo not in circuitInfo:
                        circuitInfo[gateNo]={}
                    gateType=line.split("(")[0].strip().upper()
                    circuitInfo[gateNo]["gateType"]=gateType
                    if "INPUT" in gateType:
                        circuitInfo[gateNo]["isInput"]=True
                        circuitInfo[gateNo]["isOutput"]=False
                    if "OUTPUT" in gateType:
                        circuitInfo[gateNo]["isInput"]=False
                        circuitInfo[gateNo]["isOutput"]=True
                        outputsforAT.append(gateNo)
#     except:
#         print("Cannot read the file")

In [7]:
def addAllNodes():
    for key in circuitInfo:
        allNodes.append(key)

In [8]:
def topSortRecursive(v,visited,SortOrder):
    visited[int(v)]=True
    if "fanOut" in circuitInfo[v]:
        for i in circuitInfo[v]["fanOut"]:
            if visited[int(i)]==False:
                topSortRecursive(i,visited,SortOrder)
    SortOrder.insert(0,v)

In [9]:
# //Topsort



def TopSort():
    boolVectorSize=600000
    visited=[False]*boolVectorSize

    for i in allNodes:
        if visited[int(i)] == False:
            topSortRecursive(i,visited,SortOrder)



In [11]:
NLDM_file_name="NLDM_lib_max2Inp"
delayModelFileParsing(NLDM_file_name)
# NLDM=open(NLDM_file_name,'r')
circuitFile="c17.isc"
outputsforAT=[]
circuitFileParsing(circuitFile)
allNodes=[]
addAllNodes()
SortOrder=[]
TopSort()

Parsing Circuit file : c17.isc


In [12]:
SortOrder

['5', '4', '3', '9', '11', '2', '10', '7', '1', '8', '6']

In [14]:
# outputLoad=0.00
# loopNode=''
# Nodedelay=0.00
# maxDelay=0.00
# nodeOutputSlew=0.00
# maxDelayPosition=-1

# //Forward Traversal
arrivalTimeforInputNode=0.00
outputSlewforInputNode=0.02
inverter_OutputLoad=4*1.700230
for i in SortOrder:
    currentNode=i
    if "isInput" in circuitInfo[currentNode]:
        if(circuitInfo[currentNode]["isInput"]==True):
            circuitInfo[currentNode]["arrivalTime"]=arrivalTimeforInputNode
            circuitInfo[currentNode]["outputSlewFinal"]=outputSlewforInputNode
    elif "isOutput" in circuitInfo[currentNode]:
        if(circuitInfo[currentNode]["isOutput"]==True):
            faninSize=len(circuitInfo[currentNode]["fanIn"])
            multiplier=double(faninSize)/2.0;
            if len(circuitInfo[currentNode]["fanOut"])==0:
                circuitInfo[currentNode]["outputSlewFinal"]=inverter_OutputLoad
            else:
                outputLoad=sumOfCapacitance()
        


['10']

In [69]:
circuitInfo

{'1': {'gateType': 'INPUT',
  'isInput': True,
  'isOutput': False,
  'fanOut': ['8']},
 '2': {'gateType': 'INPUT',
  'isInput': True,
  'isOutput': False,
  'fanOut': ['10']},
 '3': {'gateType': 'INPUT',
  'isInput': True,
  'isOutput': False,
  'fanOut': ['8', '9']},
 '4': {'gateType': 'INPUT',
  'isInput': True,
  'isOutput': False,
  'fanOut': ['9']},
 '5': {'gateType': 'INPUT',
  'isInput': True,
  'isOutput': False,
  'fanOut': ['11']},
 '6': {'gateType': 'NAND',
  'isInput': False,
  'isOutput': True,
  'fanIn': ['8', '10']},
 '7': {'gateType': 'NAND',
  'isInput': False,
  'isOutput': True,
  'fanIn': ['10', '11']},
 '8': {'gateType': 'NAND', 'fanIn': ['1', '3'], 'fanOut': ['6']},
 '9': {'gateType': 'NAND', 'fanIn': ['3', '4'], 'fanOut': ['10', '11']},
 '10': {'gateType': 'NAND', 'fanIn': ['2', '9'], 'fanOut': ['6', '7']},
 '11': {'gateType': 'NAND', 'fanIn': ['9', '5'], 'fanOut': ['7']}}

In [5]:
f1=int(input("number1"))
f2=int(input("numb2"))
f3=int(input("numb3"))
    if f1>f2 and f1>f3:
    print(f1,"is the largest")
elif (f2>f3):
    print(f2,"is the largest")
else:
    print(f3,"is the largest")

IndentationError: unexpected indent (1497242639.py, line 4)

In [9]:
#Question 3 Code

import numpy as np
import math
from scipy.linalg import solve


Vt=0.025
I0=(2*pow(10,-9))
Vk=0.2
I=I0*(math.exp(Vk/Vt)-1)

# Assuming knee voltage of 0.33V
Vknee=0.2
deltaV=-1
deltaI=-1
Slno=1

while((deltaV!=0)and(deltaI!=0)):
    a=(I0/Vt)*(math.exp(Vk/Vt))
    b=I-(I0/Vt)*(math.exp(Vk/Vt))*(Vk)
    print("alpha",a)
    print("beta",b)
    Solved=solve(np.array([[a,-a,1],[-a,a+1,0],[1,0,0]]),np.array([-b,b,1]))
    Iold=I
    Vkold=Vk
    Vk=Solved[0]-Solved[1]
    
    if Vk>Vknee:
        I=(a*Vk)+b
        Vk=Vt*(math.log((I/I0)+1))
    else:
        I=I0*(math.exp(Vk/Vt)-1)
    
    deltaV=Vk-Vkold
    deltaI=I-Iold

    print(str(Slno)+". New Voltage : "+str(round(Vk,3))+"V\n")
    print(str(Slno)+". New Current : "+str(round(I,3))+"A\n")
    Slno=Slno+1


alpha 0.00023847663896333827
beta -4.17354118185842e-05
1. New Voltage : 0.287V

1. New Current : 0.0A

alpha 0.007867852805775314
beta -0.0020645795102999407
2. New Voltage : 0.372V

2. New Current : 0.006A

alpha 0.23031889726636284
beta -0.07988005863113543
3. New Voltage : 0.448V

3. New Current : 0.122A

alpha 4.891051951819132
beta -2.0699722525610285
4. New Voltage : 0.482V

4. New Current : 0.479A

alpha 19.155014989600097
beta -8.760452134411013
5. New Voltage : 0.484V

5. New Current : 0.516A

alpha 20.62923376809726
beta -9.472917529910832
6. New Voltage : 0.484V

6. New Current : 0.516A

alpha 20.631921409810147
beta -9.474218886937022
7. New Voltage : 0.484V

7. New Current : 0.516A

alpha 20.631921417903857
beta -9.474218890856012
8. New Voltage : 0.484V

8. New Current : 0.516A

alpha 20.63192141790393
beta -9.474218890856045
9. New Voltage : 0.484V

9. New Current : 0.516A

alpha 20.631921417904078
beta -9.47421889085612
10. New Voltage : 0.484V

10. New Current : 0.516